# Mapboxgl Python Library

https://github.com/mapbox/mapboxgl-jupyter

## Example with Match-type Color Assignment (Categorical Data)

In [8]:
import pandas as pd
import sys
import os

from mapboxgl.viz import *
from mapboxgl.utils import *

## Set your Mapbox access token.  
Set a  `MAPBOX_ACCESS_TOKEN` environment variable or copy/paste your token  
If you do not have a Mapbox access token, sign up for an account at https://www.mapbox.com/  
If you already have an account, you can grab your token at https://www.mapbox.com/account/

In [15]:
# Must be a public token, starting with `pk`
token = os.getenv('MAPBOX_ACCESS_TOKEN')

In [10]:
# Load data from sample csv
data_url = 'https://raw.githubusercontent.com/mapbox/mapboxgl-jupyter/master/examples/cdec.csv'
df = pd.read_csv(data_url)

# Convert Elevation series to float
df['Elevation (feet)'] = df['Elevation (feet)'].astype(float)

# Clean up by dropping null rows
df = df.dropna(axis=1, how='all')

df.head(2)

,STATIONNAME,CDEC ID,USGS ID,USACE ID,CNRFC ID,lat,lon,Elevation (feet),RIVER_BASIN,COUNTY,OPERATOR,Gage Type
0,EL RIO (RIO MESA HIGH SCHOOL),ELH,NaN,NaN,NaN,34.2523,-119.143,0.0,VENTURA R,VENTURA,Ventura County,temp
1,GOLDEN GATE,GGT,NaN,NaN,NaN,37.8100,-122.474,0.0,SF BAY,SAN FRANCISCO,National Weather Service,temp


In [11]:
# Create geojson data object
df_to_geojson(
      df.fillna(''), 
      filename="cdec.geojson",
      properties=['CDEC ID', 'CNRFC ID', 'Gage Type', 'Elevation (feet)'],
      precision=4
) 

{'feature_count': 2353, 'filename': 'cdec.geojson', 'type': 'file'}

In [12]:
# Assign color stops
category_color_stops = [
    ['reservoir', 'rgb(211,47,47)'],  
    ['river', 'rgb(81,45,168)'], 
    ['snow', 'rgb(2,136,209)'], 
    ['precip', 'rgb(139,195,74)'], 
    ['temp', 'rgb(255,160,0)'], 
]

# Initialize CircleViz with Categorical Measure Data
viz = CircleViz('cdec.geojson', access_token=token, height='500px')
viz.label_property = 'CDEC ID'

# Marker color-related attributes
viz.color_property = 'Gage Type'
viz.default_color = 'grey'
viz.color_function_type = 'match'
viz.color_stops = category_color_stops

# Set center, zoom and render map
viz.center = (-121, 38.5)
viz.zoom = 4.5
viz.show() 

## Standard linear interpolation behavior

In [13]:
# Numeric color stops from ColorBrewer
sample_color_stops = [
    [0.0, 'rgb(255,255,204)'],
    [100.0, 'rgb(255,237,160)'],
    [250.0, 'rgb(254,217,118)'],
    [500.0, 'rgb(254,178,76)'],
    [1000.0, 'rgb(253,141,60)'],
    [2000.0, 'rgb(252,78,42)'],
    [4000.0, 'rgb(227,26,28)'],
    [6000.0, 'rgb(189,0,38)'],
    [10000.0,'rgb(128,0,38)']
]

# Select temperature gage records with numeric elevation data
temperature_df = df[df['Gage Type']=='temp']
data = df_to_geojson(temperature_df, properties=['CDEC ID', 'Elevation (feet)'],)

# Test CircleViz with interval measure data
viz = CircleViz(data, access_token=token, height='400px')

# Marker color-related attributes
viz.color_property = 'Elevation (feet)'
viz.color_function_type = 'interpolate'
viz.color_stops = sample_color_stops

# Set center, zoom and render map
viz.center = (-121, 37.5)
viz.zoom = 4.5
viz.show()  

## Combination of match-type and interpolate-type color and radius assignment

In [14]:
# create geojson data object
data = df_to_geojson(df, properties=['CDEC ID', 'Elevation (feet)', 'Gage Type'],)

# Radius stops for linear interpolation
sample_radius_stops = [
    [0.0, 1.0],
    [100.0, 2.0],
    [500.0, 3.0],
    [1000.0, 4.0],
    [5000.0, 5.0],
    [10000.0, 6.0],   
]

# Initialize Graduated Circle Visualization 
viz2 = GraduatedCircleViz(data, access_token=token, height='400px')
# viz2.label_property = 'CDEC ID'

# Marker color-related attributes
viz2.color_function_type = 'match'
viz2.color_stops = category_color_stops
viz2.color_property = 'Gage Type'
viz2.default_color = 'grey'
viz2.opacity = 0.5

# Marker radius-related attributes
viz2.radius_property = 'Elevation (feet)'
viz2.radius_stops = sample_radius_stops
viz2.radius_function_type = 'interpolate'

# Set center, zoom and render map
viz2.center = (-121, 37.5)
viz2.zoom = 4.5
viz2.show()